FaceValue Bioinstrumentation Project | Coded by Cata Santos Anzola | Finished 26/11/2025

cata comment: please run the arduino code before running this code! This code will receive the serial output from the arduino (stored in the SD card), and put it through our EMG-classifying model. The outcome of the model will then print out a string that will be sent to the arduino to show on the LCD.

Step 0. Import libraries

In [49]:
# initialize all libraries
import numpy as np
import joblib
from keras.models import load_model
import serial
import sys
import inspect
from pyfirmata2 import Arduino
import time
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime

Step 1. Connect with Arduino

In [50]:
# auto detect COM port so we don't have to change it everytime
PORT = Arduino.AUTODETECT
PINS = [1, 2, 3, 4, 5]  # A1 through A5
SAMPLING_RATE = 10  # Hz
DATA_DIR = "EMG_recordings"
os.makedirs(DATA_DIR, exist_ok=True)

Step 2. Collect EMG Signals and Save as CSV into SD Card

In [76]:
class MultiSensorRecorder:
    def __init__(self):
        self.recording = False
        self.recording_count = 0
        self.data = {pin: [] for pin in PINS}
        
        print("Connecting to Arduino...")
        self.board = Arduino(PORT)
        
        # Setup callbacks
        self.setup_callbacks()
        
        # Start sampling
        self.board.samplingOn(1000 / SAMPLING_RATE)
        
        print("Connected! Ready to record from pins A1-A5")
        
    def setup_callbacks(self):
        """Setup callback functions for each pin"""
        def make_callback(pin):
            def callback(value):
                if self.recording:
                    timestamp = time.time() - self.start_time
                    voltage = value * 5.0
                    self.data[pin].append([timestamp, voltage])
                    
                    # Print update from first pin only
                    if pin == PINS[0] and len(self.data[pin]) % 10 == 0:
                        print(f"  {timestamp:.1f}s: {voltage:.3f}V", end='\r')
            return callback
        
        # Register callbacks
        for pin in PINS:
            self.board.analog[pin].register_callback(make_callback(pin))
    
    def start_recording(self):
        """Start a new recording session"""
        if self.recording:
            print("Already recording!")
            return
            
        self.recording = True
        self.recording_count += 1
        self.start_time = time.time()
        
        # Clear previous data
        self.data = {pin: [] for pin in PINS}
        
        # Enable reporting for all pins
        for pin in PINS:
            self.board.analog[pin].enable_reporting()
        
        print(f"\nRecording #{self.recording_count} started...")
    
    def stop_recording(self):
        """Stop recording and save data"""
        if not self.recording:
            print("Not recording!")
            return
            
        self.recording = False
        
        # Disable reporting
        for pin in PINS:
            self.board.analog[pin].disable_reporting()
        
        time.sleep(0.5)  # Let any pending callbacks finish
        
        # Save data
        self.save_data()
        
        return self.data
    
    def save_data(self):
        """Save collected data to CSV files"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print(f"\nSaving recording #{self.recording_count}...")
        
        for pin in PINS:
            if self.data[pin]:
                df = pd.DataFrame(self.data[pin], columns=["Time", "Voltage"])
                filename = f"A{pin}.csv"
                filepath = os.path.join(DATA_DIR, filename)
                df.to_csv(filepath, index=False)
                print(f"  A{pin}: {len(df)} samples -> {filename}")
            else:
                print(f"  A{pin}: No data collected")
    
    def record_for_duration(self, duration):
        """Record for a specific duration"""
        print(f"\nRecording for {duration} seconds...")
        self.start_recording()
        time.sleep(duration)
        return self.stop_recording()
    
    def close(self):
        """Clean up"""
        self.board.samplingOff()
        time.sleep(0.1)
        self.board.exit()
        print("Arduino connection closed")

def simple_automatic_recording():
    """Simple version: Record for fixed duration when prompted"""
    
    recorder = MultiSensorRecorder()
    recording_num = 0
    
    try:
        print("\n" + "="*50)
        print("AUTOMATIC MULTI-SENSOR RECORDER")
        print("="*50)
        print("Each recording creates 5 CSV files (A1-A5)")
        print("Press Ctrl+C to quit")
        print("="*50)
        
        while True:
            recording_num += 1
            
            # Get duration
            try:
                duration_input = input(f"\nRecording #{recording_num} - Duration (seconds, Enter for 5): ")
                duration = float(duration_input) if duration_input else 5
            except:
                duration = 5
                print(f"Using default: {duration} seconds")
            
            # Countdown
            print(f"\nStarting in...")
            for i in range(3, 0, -1):
                print(f"{i}...")
                time.sleep(1)
            
            print("RECORDING!")
            
            # Record
            recorder.record_for_duration(duration)
            
            print(f"\nRecording complete!")
            
            # Ask to continue
            cont = input("\nRecord another? (y/n): ").lower()
            if cont != 'y':
                break
                
    except KeyboardInterrupt:
        print("\n\nStopped by user")
    except Exception as e:
        print(f"\nError: {e}")
    finally:
        recorder.close()
        print(f"Data saved in: {DATA_DIR}")
# Run the program ughhhh
if __name__ == "__main__":
    simple_automatic_recording()
 

Connecting to Arduino...
Connected! Ready to record from pins A1-A5

AUTOMATIC MULTI-SENSOR RECORDER
Each recording creates 5 CSV files (A1-A5)
Press Ctrl+C to quit

Starting in...
3...
2...
1...
RECORDING!

Recording for 5 seconds...

Recording #1 started...
  4.8s: 1.403V
Saving recording #1...
  A1: 51 samples -> A1.csv
  A2: 51 samples -> A2.csv
  A3: 51 samples -> A3.csv
  A4: 51 samples -> A4.csv
  A5: 51 samples -> A5.csv

Recording complete!
Arduino connection closed
Data saved in: EMG_recordings


Step 3. Visualize graphs to make sure we are on the right track (and also importing the right csv files)

In [81]:
path = "C:/Users/catas/OneDrive - Clemson University/Uni/Bioinstrumentation Lab/FaceValue/FaceValue_ML/EMG_recordings"
files = glob.glob(path + '/*.csv')
a = []
# loop through list of files and read each one into a dataframe and append to list
for f in files:

    # get filename
    Pin = os.path.basename(f)

    # read in csv
    temp_df = pd.read_csv(f)

    # create new column with filename
    temp_df['EMG Pin'] = Pin

    # data cleaning to remove the nasty csv ending
    temp_df['EMG Pin'] = temp_df['EMG Pin'].replace('.csv', '', regex=True)

    # append df to list
    a.append(temp_df)

# concatenate our list of dataframes into one!
df = pd.concat(a, axis=0)
print(df.shape)
df.head()


(510, 3)


,Time,Voltage,EMG Pin
0,0.004151,2.5220,A1
1,0.040843,2.0820,A1
2,0.142427,1.9210,A1
3,0.242058,1.8575,A1
4,0.343820,1.8135,A1


Step 2. Receive SD Card Data

In [ ]:
# cata comment: this part of the code will run after getting the Arduino to ouput the Serial.monitor data into the SD card.
# We will read the data from the SD card, process it, and then input it into the model to get the prediction.
#forehead = 


Step 3. Run EMG signal into ML model for classification

In [ ]:
#with our SD card data now formatted, we can load the model and get the prediction:

model = load_model('C:/Users/catas/OneDrive - Clemson University/Uni/Bioinstrumentation Lab/FaceValue/EMGClassifier.hdf5') # load in the .hd5 model that was trained from ALS/control datasets
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #compile the model to have an outcome based on accuracy and loss
outcome=model.predict(data) #get the prediction from the model based on the data from SD card
print(outcome) #print this so we can see it as we run & trouble shoot, comment out later if needed


#cata comment: this inequality isn't final as we need to see how each EMG weighs into the prediction.
if outcome != 0:
    stringa = 'ALS Suspected'
else:
    stringa = 'ALS Not Suspected'

Step 4. output string from model result to arduino

           ,....,
        ,;;:o;;;o;;,
      ,;;o;'''''';;;;,
     ,;:;;        ;;o;,
     ;o;;          ;;;;        Happy Holidays <3
     ';;;,  _  _  ,;;;'
      ';o;;/_\/_\;;o;'
         ';;\_\/_/;;'
           '//\\'
           //  \\
          |/    \|